# Performing several tasks at the same time on the GPU

---
**Requirements:**

- [Get started](./Get_started.ipynb)
- [Atomic operations](./Atomic_operations.ipynb)
- [Manual building](./Manual_building.ipynb)
- [Data management](./Data_management.ipynb)

---

This part describes how to overlap several kernels on the GPU and/or how to overlap kernels with data transfers.
This feature is called asynchronism and will give you the possibility to get better performance when it is possible to implement it.

On the GPU you can have several execution threads (called _streams_ or _activity queue_) running at the same time independently.
A _stream_ can be viewed as a pipeline that you feed with kernels and data transfers that have to be executed in order.

So as a developer you can decide to activate several streams if your code is able to withstand them.
OpenACC gives you the possibility to manage streams with the tools:

- _async_ clause
- _wait_ clause or directive

 By default, only one stream is created.

## _async_ clause

Some directives accept the clause _async_ to run on another stream than the default one.
You can specify an integer (which can be a variable) to have several streams concurrently.

If you omit the optional integer then a "default" extra stream is used.

The directives which accept _async_ are:

- the compute constructs: `acc parallel`, `acc kernels`, `acc serial`
- the unstructured data directives: `acc enter data`, `acc exit data`, `acc update`
- the `acc wait` directive

For example we can create 2 streams to allow data transfers and kernel overlap.

```c
int stream1=1;
int stream2=2;
#pragma acc enter data copyin(array[:size]) async(stream1)
// Some stuff
#pragma acc parallel async(stream2)
{
    // A wonderful kernel
}
```

## _wait_ clause

Running fast is important but having correct results is surely more important.

If you have a kernel that needs the result of another kernel or that a data transfer is complete then you have to wait for the operations to finalize.
You can add the _wait_ clause (with an optional integer) to the directives:

- the compute constructs: `acc parallel`, `acc kernels`, `acc serial`
- the unstructured data directives: `acc enter data`, `acc exit data`, `acc update`

This example implements 2 streams but this time the kernel needs the data transfer on stream1 to complete before being executed.

```c
int stream1=1;
int stream2=2;
#pragma acc enter data copyin(array[:size]) async(stream1)
// Some stuff
#pragma acc parallel async(stream2) wait(stream1)
{
    // A wonderful kernel
}       
```

Furthermore you can wait for several streams to complete by giving a comma-separated list of integers as clause arguments

This example implements 2 streams but this time the kernel needs the data transfer on stream1 to complete before being executed.

```c
int stream1=1;
int stream2=2;
int stream3=3;
#pragma parallel loop async(stream3)
for (int i=0; i <size; ++i)
{
    // Kernel launched on stream3
}
#pragma acc enter data copyin(array[:size]) async(stream1)
// Some stuff
#pragma acc parallel async(stream2) wait(stream1, stream3)
{
    // A wonderful kernel
}    
```


If you omit the clause options, then the operations will wait until all asynchronous operations fulfill.

```c
#pragma acc parallel wait
{
    // A wonderful kernel
}    
```

## _wait_ directive

_wait_ comes also as a standalone directive.
```c
int stream1=1;
int stream2=2;
int stream3=3;
#pragma parallel loop async(stream3)
for (int i=0; i <size; ++i)
{
    // Kernel launched on stream3
}
#pragma acc enter data copyin(array[:size]) async(stream1)
// Some stuff

#pragma acc wait(stream3)

#pragma acc parallel async(stream2)
{
    // A wonderful kernel
}    

```

## Exercise

In this exercise you have to compute the matrix product $C = A \times B$.

You have to add directives to:

- use the program lifetime unstructured data region to allocate memory on the GPU
- perform the matrix initialization on the GPU
- perform the matrix product on the GPU
- create and analyze a profile (add the option `--profile` to idrrun)
- save the .qdrep file
- check what can be done asynchronously and implement it
- create and analyze a profile (add the option `--profile` to idrrun)
- save the .qdrep file

Your solution is considered correct if no implicit action are done.

Example stored in: `../../examples/C/async_async_exercise.c`

In [ ]:
%%idrrun -a 
#include <stdio.h>
#include <stdlib.h>
double* create_mat(int dim, int stream)
{
    double* mat = (double*) malloc(dim*dim*sizeof(double));
    return mat;
}

void init_mat(double* mat, int dim, double diag, int stream)
{
    for (int i=0; i<dim; ++i)
        for (int j=0; j<dim; ++j)
        {
            mat[i*dim+j] = 0.;
        }
    for (int i=0; i<dim; ++i)
        mat[i*dim+i] = diag;
}

int main(void)
{
    int dim = 5000;
    
    double* restrict A = create_mat(dim, 1);
    double* restrict B = create_mat(dim, 2);
    double* restrict C = create_mat(dim, 3);
    
    init_mat(A, dim, 6.0, 1);
    init_mat(B, dim, 7.0, 2);
    init_mat(C, dim, 0.0, 3);

    for (int i=0; i<dim; ++i)
        for (int k=0; k<dim; ++k)
            for (int j=0; j<dim; ++j)
            {
                C[i*dim+j] += A[i*dim+k] * B[k*dim+j];
            }
    printf("Check that value is equal to 42.: %f\n", C[0]);
    return 0;
}


### Solution

Example stored in: `../../examples/C/async_async_solution.c`

In [ ]:
%%idrrun -a --profile
#include <stdio.h>
#include <stdlib.h>
double* create_mat(int dim, int stream)
{
    double* mat = (double*) malloc(dim*dim*sizeof(double));
    #pragma acc enter data create(mat[0:dim*dim]) async(stream)
    return mat;
}

void init_mat(double* mat, int dim, double diag, int stream)
{
    #pragma acc parallel loop present(mat[0:dim*dim]) async(stream)
    for (int i=0; i<dim; ++i)
        #pragma acc loop
        for (int j=0; j<dim; ++j)
        {
            mat[i*dim+j] = 0.;
        }
    #pragma acc parallel loop present(mat[0:dim*dim]) async(stream)
    for (int i=0; i<dim; ++i)
        mat[i*dim+i] = diag;
}

int main(void)
{
    int dim = 5000;
    
    double* restrict A = create_mat(dim, 1);
    double* restrict B = create_mat(dim, 2);
    double* restrict C = create_mat(dim, 3);
    
    init_mat(A, dim, 6.0, 1);
    init_mat(B, dim, 7.0, 2);
    init_mat(C, dim, 0.0, 3);

    #pragma acc parallel present(A[:dim*dim], B[:dim*dim], C[:dim*dim]) wait(1,2,3)
    {
    #pragma acc loop gang vector collapse(3)
    for (int i=0; i<dim; ++i)
        for (int k=0; k<dim; ++k)
            for (int j=0; j<dim; ++j)
            {
                #pragma acc atomic update
                C[i*dim+j] += A[i*dim+k] * B[k*dim+j];
            }
    }
    #pragma acc exit data delete(A[:dim*dim], B[:dim*dim]) copyout(C[:dim*dim])
    printf("Check that value is equal to 42.: %f\n", C[0]);
    return 0;
}

In an ideal world, the solution would produce a profile like this one:

<img src="../../pictures/async.png" style="float:none"/>

### Comments

- Several threads will update the same memory location for C so you have to use an `acc atomic update`
- `collapse` is used to fuse the 3 loops. It helps the compiler to generate a more efficient code

## Advanced NVIDIA compiler option to use Pinned Memory: `-gpu=pinned`

If you look at the profiles of your code (at this point "if" should be "when"), you can see that the memory transfers occurs in chunks of more or less constant size.
Even though you have a large memory block it will be split into several smaller pieces which have the size of a memory page.

Memory not pinned:

<img alt="Nsight output unpinned memory" src="../../pictures/NSight-matmul_not_pinned.png" style="float:none"/>

Memory pinned:

<img alt="Nsight output pinned memory" src="../../pictures/NSight-matmul_pinned.png" style="float:none"/>

Usually the transfer time is reduced when pinned memory is used.
It can also cause some segmentation faults. Do your testing!

### Bonus

You can launch the exercise with `%%idrrun -a --profile --accopts "cc70,pinned"` to test the effect of pinned memory.
You can save a profile to compare the 3 versions.